In [2]:
pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.9 MB/s eta 0:00:00


In [3]:
pip install -U qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 83.4 MB/s eta 0:00:00


In [22]:
from qiskit_aer import Aer

In [23]:
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import numpy as np

In [24]:
size = 5
state_values = [22, 17, 27, 12]
target_amplitude = 0.5 / np.sqrt(len(state_values))

In [25]:
qc = QuantumCircuit(size)

In [26]:

qc.x([4, 2, 1])   # |10110⟩
qc.x([4, 0])       # |10001⟩
qc.x([4, 3, 1, 0]) # |11011⟩
qc.x([3, 2])       # |01100⟩

In [27]:

qc.rz(np.pi/4, 4)
qc.rz(np.pi/8, 3)
qc.rz(np.pi/6, 2)
qc.rz(np.pi/12, 1)
qc.rz(np.pi/10, 0)

In [28]:

qc.cx(4, 0)
qc.cx(4, 1)
qc.cx(4, 2)
qc.cx(4, 3)

In [32]:
from qiskit import transpile
coupling_map = [[0, 4], [1, 4], [2, 4], [3, 4], [0, 1], [2, 3]]
optimized_circuit = transpile(qc, basis_gates=['x', 'h', 'rz', 'cx'], coupling_map=coupling_map, optimization_level=3)
optimized_circuit.draw()

┌──────────┐   ┌───┐                  ┌───┐          
q_1 -> 0 ┤ Rz(π/12) ├───┤ H ├───────────────■──┤ H ├──────────
         ├──────────┤   ├───┤               │  ├───┤          
q_0 -> 1 ┤ Rz(π/10) ├───┤ H ├──────────■────┼──┤ H ├──────────
         ├─────────┬┘   ├───┤          │    │  └───┘     ┌───┐
q_3 -> 2 ┤ Rz(π/8) ├────┤ H ├──────────┼────┼─────────■──┤ H ├
         ├─────────┤    ├───┤          │    │         │  ├───┤
q_2 -> 3 ┤ Rz(π/6) ├────┤ H ├──────────┼────┼────■────┼──┤ H ├
         └──┬───┬──┘ ┌──┴───┴──┐┌───┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐├───┤
q_4 -> 4 ───┤ X ├────┤ Rz(π/4) ├┤ H ├┤ X ├┤ X ├┤ X ├┤ X ├┤ H ├
            └───┘    └─────────┘└───┘└───┘└───┘└───┘└───┘└───┘

In [12]:
simulator = Aer.get_backend('statevector_simulator')

In [13]:
transpiled_qc = transpile(qc, simulator)

In [15]:
result = simulator.run(transpiled_qc).result()
statevector = result.get_statevector()

In [ ]:
print("Generated State Vector:", statevector)

In [17]:
print("Circuit Depth:", optimized_circuit.depth())

Circuit Depth: 8


In [18]:
print(optimized_circuit)

         ┌─────────┐    ┌───┐                                 ┌───┐
q_3 -> 0 ┤ Rz(π/8) ├────┤ H ├──────────────────────────────■──┤ H ├
         ├─────────┤    ├───┤                              │  ├───┤
q_2 -> 1 ┤ Rz(π/6) ├────┤ H ├─────────────────────────■────┼──┤ H ├
         ├─────────┴┐   ├───┤                  ┌───┐  │    │  └───┘
q_1 -> 2 ┤ Rz(π/12) ├───┤ H ├───────────────■──┤ H ├──┼────┼───────
         ├──────────┤   ├───┤               │  ├───┤  │    │       
q_0 -> 3 ┤ Rz(π/10) ├───┤ H ├──────────■────┼──┤ H ├──┼────┼───────
         └──┬───┬───┘┌──┴───┴──┐┌───┐┌─┴─┐┌─┴─┐└───┘┌─┴─┐┌─┴─┐┌───┐
q_4 -> 4 ───┤ X ├────┤ Rz(π/4) ├┤ H ├┤ X ├┤ X ├─────┤ X ├┤ X ├┤ H ├
            └───┘    └─────────┘└───┘└───┘└───┘     └───┘└───┘└───┘


In [19]:
target_states = [22, 17, 27, 12]
for state in target_states:
    print(f"Amplitude of state |{np.binary_repr(state, width=5)}⟩:", statevector[state])

Amplitude of state |10110⟩: -0j
Amplitude of state |10001⟩: -0j
Amplitude of state |11011⟩: -0j
Amplitude of state |01100⟩: 0j


In [21]:
print("Circuit Depth:", optimized_circuit.depth())

Circuit Depth: 8
